In [87]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
# from docplex.mp.model import Model
from gurobipy import GRB, Model

In [88]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('randomized_graph.csv')

In [89]:
#row with name = 'root_cross_right' to dict
static_node = static[static['name'] == 'root_cross_right'].to_dict('records')[0]

In [90]:
static_node

{'name': 'root_cross_right',
 'number': -1,
 'x': 10.0,
 'y': 1.0,
 'z': 0.0,
 'parent': 'root',
 'distance_to_parent': 10}

In [91]:
detected_node = detected[detected['name'] == 'root_cross_right'].to_dict('records')[0]

In [92]:
detected_node['x']=12

In [93]:
detected_node

{'name': 'root_cross_right', 'number': -1, 'x': 12, 'y': 1.0, 'z': 7.55}

In [94]:
# quadratic_model = Model(name='quadratic')
quadratic_model = Model('quadratic')

In [95]:
# x = quadratic_model.continuous_var(name = 'x')
# y = quadratic_model.continuous_var(name = 'y')
# z = quadratic_model.continuous_var(name = 'z')

x = quadratic_model.addVar(name = 'x', vtype=GRB.CONTINUOUS)
y = quadratic_model.addVar(name = 'y', vtype=GRB.CONTINUOUS)
z = quadratic_model.addVar(name = 'z', vtype=GRB.CONTINUOUS)

In [96]:
# quadratic_model.add(
#     x**2 + y**2 + z**2 <= static_node["distance_to_parent"] ** 2
# )  # to dziala tylko dla tego noda bo parent ma pos 0,0,0

quadratic_model.addQConstr(
    x**2 + y**2 + z**2 <= static_node["distance_to_parent"] ** 2
)  # to dziala tylko dla tego noda bo parent ma pos 0,0,0

<gurobi.QConstr Not Yet Added>

In [97]:
obj_fn = (x-static_node["x"])**2 + (y-static_node["y"])**2 + (z-static_node["z"])**2

In [98]:
# quadratic_model.set_objective('min', obj_fn)

quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [99]:
# quadratic_model.print_information()

quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8365U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xb640c0be
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+02, 1e+02]
Presolve time: 0.05s
Presolved: 8 rows, 9 columns, 12 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.400e+01
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          

In [100]:
for v in quadratic_model.getVars():
    print('%s %g' % (v.varName, v.x))

x 9.95037
y 0.99503
z 0.00128799
